<a href="https://colab.research.google.com/github/ARAN1218/piedpiper-python/blob/main/PPP%E2%91%A4_AI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 人工知能 ~人類最悪にして最後の発明~

## 早速やっていこう！...あれ？

In [ ]:
# 外部ライブラリを召喚(インポート)しよう→pyファイルをインストールしている
# ライブラリをインポートする際は(特にノートブック形式だと)最初の段階で分析に使うライブラリを全部インポートする慣習がある
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
from sklearn.datasets import load_boston

In [ ]:
# ライブラリを使って簡単にデータをダウンロードしてみよう
boston = load_boston()
boston_data = boston.data
columns_name = boston.feature_names
df = pd.DataFrame(boston_data, columns=columns_name)
display(df)
# 何かこのデータセットは倫理的な問題があるとかで将来的に廃止するそうです

## じゃあ別のデータセットでやろうか

In [ ]:
# 警告文に記載されている推奨ライブラリをインポート
from sklearn.datasets import fetch_california_housing
housing = fetch_california_housing()

# ライブラリを用いて、説明変数(予測の材料)を簡単にロードする
housing_data = housing.data
housing_data_names = housing.feature_names
df_data = pd.DataFrame(housing_data, columns=housing_data_names)

# 目的変数(予測対象)も簡単にロードできる
housing_target = housing.target
housing_target_names = housing.target_names
df_target = pd.DataFrame(housing_target, columns=housing_target_names)

# 作成したデータフレームを見てみる
display(df_data)
display(df_target)

# 説明変数と目的変数のデータフレームを結合する
df = pd.concat([df_data, df_target], axis=1)
display(df)

### 説明変数の意味


MedInc：ブロックグループの収入の中央値

HouseAge：ブロックグループの築年数の中央値

AveRooms：1世帯あたりの平均部屋数

AveBedrms：1世帯あたりの平均寝室数

Population：ブロックグループの人口

AveOccup：平均世帯員数

Latitude：ブロックグループの緯度

Longitude：ブロックグループの経度

### 目的変数の意味

MedHouseVal：対象ブロックの住宅価格の中央値

## 早速機械学習...する前に

In [ ]:
# データを可視化して、目的変数と説明変数の関係を分析しよう
# データの特性も分からないのにいきなり機械学習モデル作るのは愚か極まりない

def scatter(data):
  for data_name in housing.feature_names:
    plt.title(data_name + " vs. MedHouseVal")
    plt.xlabel(data_name)
    plt.ylabel('MedHouseVal')
    plt.scatter(df[data_name], df['MedHouseVal']) # plt.scatter(x, y)
    plt.show()
    print('相関係数：',np.corrcoef(df[data_name], df['MedHouseVal'])[0][1])

scatter(df) # 散布図からどんな関係があるかを考えてみよう！

In [ ]:
# 上記の作業はseabornライブラリを用いて2行で実現できる
# →有用なライブラリを検索して活用する能力も必要！
sns.pairplot(df)
plt.show()

In [ ]:
# ヒートマップ作成も一行で完結
sns.heatmap(df.corr())
plt.show()

# 各種引数で調整可
sns.heatmap(df.corr(), annot=True, fmt="1.1f", cmap='Oranges')
plt.show()

## まずは単回帰分析から

In [ ]:
# データセットを学習データとテストデータに分ける→モデルの評価に使うため！
from sklearn.model_selection import train_test_split
df_MedInc = df_data[['MedInc']] # MedIncカラムだけ取り出す
x_train, x_test, y_train, y_test = train_test_split(df_MedInc, df_target, shuffle=True, random_state=71)


# 機械学習ライブラリをインポート
from sklearn.linear_model import LinearRegression

# 機械学習モデルのインスタンスを生成
model = LinearRegression()

# モデルを学習させる
model.fit(x_train, y_train)

# テストデータを使って、モデルに予測させる
pred = model.predict(x_test)

# 予測結果を出力させる
df_pred = pd.DataFrame(pred, columns=['pred'])
df_true = y_test.reset_index(drop=True).rename(columns={'MedHouseVal': 'true'})
df = pd.concat([df_pred, df_true], axis=1)
display(df)

In [ ]:
# モデルの評価を行うため、MAE/RMSE/R^2の計算と残差プロットを行う

# MAE/RMSE/R^2の計算
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
print("MAE:", mean_absolute_error(df['true'], df['pred'])) # 平均絶対誤差...小さい程良い
print("RMSE:", np.sqrt(mean_absolute_error(df['true'], df['pred']))) # 平均平方二乗誤差...小さい程良い
print("R^2:", model.score(x_train, y_train)) # 決定係数...大きい程良い

# 傾きと切片を見てみる
print('傾き(回帰係数) = ', model.coef_[0])
print('切片 = ', model.intercept_)

# 残差プロット...予測値と真の値がどれだけ離れているかを可視化
plt.scatter(df['pred'], df['pred'] - df['true'], c='green', marker='s', label='Test Data')
plt.xlabel('Predicted Values')
plt.ylabel('Residuals')
plt.legend(loc='upper left')
plt.hlines(y=0, xmin=-10, xmax=20, lw=2, color='red')
plt.show()

## お待ちかねの多変量解析(重回帰分析)

In [ ]:
# 次は全部の変数をぶち込んでみる
x_train, x_test, y_train, y_test = train_test_split(df_data, df_target, shuffle=True, random_state=71)
model = LinearRegression()
model.fit(x_train, y_train)
pred = model.predict(x_test)
df_pred = pd.DataFrame(pred, columns=['pred'])
df_true = y_test.reset_index(drop=True).rename(columns={'MedHouseVal': 'true'})
df = pd.concat([df_pred, df_true], axis=1)
display(df)


In [ ]:
# MAE/RMSE/R^2の計算
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
print("MAE:", mean_absolute_error(df['true'], df['pred']))
print("RMSE:", np.sqrt(mean_absolute_error(df['true'], df['pred'])))
print("R^2:", model.score(x_train, y_train))

# 傾きと切片を見てみる
print('傾き(偏回帰係数) = ', model.coef_[0]) # 説明変数の係数を出力
print('切片 = ', model.intercept_)

# 残差プロット
plt.scatter(df['pred'], df['pred'] - df['true'], c='green', marker='s', label='Test Data')
plt.xlabel('Predicted Values')
plt.ylabel('Residuals')
plt.legend(loc='upper left')
plt.hlines(y=0, xmin=-10, xmax=20, lw=2, color='red')
plt.show()

# ちょっとだけ当てはまりが良くなった→モデルが複雑（説明変数が多い）だとモデルの性能が（一般的に）良くなる

## 重回帰分析の問題点

- 偏相関係数
 - 重回帰分析で説明変数として採用する変数の評価に用いる。
- 自由度調整済み重相関係数
 - 単回帰分析モデルと重回帰分析モデルをそのまま比較するのは不公平！→自由度（=学習に用いた説明変数の数）で調整してフェアな比較をする。
- 多重共線性
 - 偏回帰係数の値に極端な補正がかかってしまい、分析として適さないモデルができてしまう。
 - 説明変数同士で相関が高いと発生するので、相関の高い説明変数のペアが確認された場合にはそのどちらかだけを採用することで回避できる。
 - もしくはVIF(Variance Inflation Factors)を計算し、その値が10以下の説明変数だけを採用すれば良い！ 
- 残差分析
 - 外れ値や異常値を確認し、隠れた因子を考慮する
 - データの傾向から、分散の均質性や非線形性を確認する。
- 変数選択法
 - 全ての説明変数の組み合わせを分析するのは大変なので、回帰係数の選別をする。
 - 情報量基準等を調べてみましょー🐦
- ダミー変数
 - 質的変数(男女等)を重回帰分析に組み込むためには、質的変数を数値に変換する必要がある。
 - 例えば、男：０、女：１として変換すると、男女の区別をつけながらその影響を分析することができる！

## Webデータ分析実践

In [ ]:
# 気象庁のWebサイトから気象データを取ってくる(これをスクレイピングと言います)
url = "https://www.data.jma.go.jp/obd/stats/etrn/view/daily_s1.php?prec_no=46&block_no=47670&year=2022&month=1&day=1&view="
df_weather = pd.read_html(url, header = 0)
df_weather

In [ ]:
# リストにしまわれているので、一個にする
df_weather = df_weather[0]
df_weather.head(10)
# 汚ねぇ！！→分析に使えるデータに直します

In [ ]:
# 日本語の行を削る
df_weather = df_weather.drop(index=df.index[[0,1,2]])
df_weather.head(5)

In [ ]:
# データ型を確認
df_weather.dtypes
# 汚ねぇ！！！→数値に直す

In [ ]:
# 今回は説明変数を平均湿度、目的変数を平均気温にして単回帰分析してみる
# 小数が多いので、float型に直す
df_weather = df_weather[['湿度(％)', '気温(℃)']]
df_weather['湿度(％)'] = df_weather['湿度(％)'].astype(float)
df_weather['気温(℃)'] = df_weather['気温(℃)'].astype(float)
df_weather.dtypes

In [ ]:
# 単回帰分析をする
from sklearn.model_selection import train_test_split
df_x = df_weather[['湿度(％)']]
df_y = df_weather[['気温(℃)']]
x_train, x_test, y_train, y_test = train_test_split(df_x, df_y, shuffle=True, random_state=71)


# 機械学習ライブラリをインポート
from sklearn.linear_model import LinearRegression

# 機械学習モデルのインスタンスを生成
model = LinearRegression()

# モデルを学習させる
model.fit(x_train, y_train)

# テストデータを使って、モデルに予測させる
pred = model.predict(x_test)

# 予測結果を出力させる
df_pred = pd.DataFrame(pred, columns=['pred'])
df_true = y_test.reset_index(drop=True).rename(columns={'気温(℃)': 'true'})
df = pd.concat([df_pred, df_true], axis=1)
display(df)

In [ ]:
# モデルの評価を行うため、MAE/RMSE/R^2の計算と残差プロットを行う

# MAE/RMSE/R^2の計算
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
print("MAE:", mean_absolute_error(df['true'], df['pred'])) # 平均絶対誤差...小さい程良い
print("RMSE:", np.sqrt(mean_absolute_error(df['true'], df['pred']))) # 平均平方二乗誤差...小さい程良い
print("R^2:", model.score(x_train, y_train)) # 決定係数...大きい程良い

# 傾きと切片を見てみる
print('傾き(回帰係数) = ', model.coef_[0])
print('切片 = ', model.intercept_)

# 残差プロット...予測値と真の値がどれだけ離れているかを可視化
plt.scatter(df['pred'], df['pred'] - df['true'], c='green', marker='s', label='Test Data')
plt.xlabel('Predicted Values')
plt.ylabel('Residuals')
plt.legend(loc='upper left')
plt.hlines(y=0, xmin=-10, xmax=20, lw=2, color='red')
plt.show()